# Importing Libraries


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
from collections import defaultdict
from pprint import pprint

In [2]:
company_data=pd.read_csv('/content/drive/MyDrive/SEM9/Information Retrieval/Financial_Search_Engine/package datasets/Merged_Stock_Data', index_col=0)

In [3]:
company_data

,Symbol,Company_Name,Current_Price,Earnings_per_Share,Growth_Rate,Intrinsic_Value,Long Business Summary
0,RRX,Regal Rexnord Corporation,118.2800,9.947,-0.774,0.177498,Regal Rexnord Corporation manufactures and sel...
1,APTMU,Alpha Partners Technology Merger Corp.,10.6200,0.013,0.311,0.000021,Alpha Partners Technology Merger Corp. does no...
2,WSFS,WSFS Financial Corporation,35.2350,9.927,0.047,2.967547,WSFS Financial Corporation operates as the sav...
3,FRAF,Franklin Financial Services Corporation,32.0000,6.153,-0.162,0.961493,Franklin Financial Services Corporation operat...
4,WF,Woori Financial Group Inc.,26.7500,63314.470,-0.342,121665.258441,"Woori Financial Group Inc., together with its ..."
...,...,...,...,...,...,...,...
230,WEYS,"Weyco Group, Inc.",28.5300,2.452,0.064,0.225463,"Weyco Group, Inc. designs and distributes foot..."
231,FE,FirstEnergy Corp.,36.0085,0.206,0.196,0.001411,"FirstEnergy Corp., through its subsidiaries, g..."
232,SHEL,Shell plc,65.3350,6.695,-0.808,0.121649,Shell plc operates as an energy and petrochemi...
233,TXT,Textron Inc.,75.4400,8.525,0.274,1.266532,"Textron Inc. operates in the aircraft, defense..."


#### Pre-processing

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk
import math
import string
from itertools import groupby
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

def lowercase(text):
  return text.lower()

def preprocess(text):
  text = lowercase(text)
  text = remove_punctuation(text)
  text = remove_stop_words(text)
  return text

In [6]:
documents = company_data['Long Business Summary']

In [7]:
company_symbols = company_data['Symbol']

In [8]:
documents[0]

'Regal Rexnord Corporation manufactures and sells industrial powertrain solutions, power transmission components, electric motors and electronic controls, air moving products, and specialty electrical components and systems worldwide. It operates in four segments: Commercial Systems, Industrial Systems, Climate Solutions, and Motion Control Solutions. The Commercial Systems segment provides AC and DC motors, electronic variable speed controls, fans, blowers, and precision stator and rotor kits. The Industrial Systems segment offers AC motors for industrial applications; electric alternators for prime and standby power applications to data centers, distributed energy, microgrid, rental, marine, agriculture, healthcare, mobile, and defense markets; switchgear, switchboards, and control systems for power generation systems; and transfer switch power solutions for residential, commercial, industrial, and critical applications. The Climate Solutions segment provides fractional horsepower mo

In [9]:
process_docs = [preprocess(x).lower() for x in documents]

In [10]:
process_docs[0]

'regal rexnord corporation manufactures sells industrial powertrain solutions power transmission components electric motors electronic controls air moving products specialty electrical components systems worldwide operates four segments commercial systems industrial systems climate solutions motion control solutions commercial systems segment provides ac dc motors electronic variable speed controls fans blowers precision stator rotor kits industrial systems segment offers ac motors industrial applications electric alternators prime standby power applications data centers distributed energy microgrid rental marine agriculture healthcare mobile defense markets switchgear switchboards control systems power generation systems transfer switch power solutions residential commercial industrial critical applications climate solutions segment provides fractional horsepower motors electronic variable speed controls blowers use residential light commercial air moving applications fractional horse

In [11]:
company_data['preprocessed_description'] = process_docs

In [12]:
company_data

,Symbol,Company_Name,Current_Price,Earnings_per_Share,Growth_Rate,Intrinsic_Value,Long Business Summary,preprocessed_description
0,RRX,Regal Rexnord Corporation,118.2800,9.947,-0.774,0.177498,Regal Rexnord Corporation manufactures and sel...,regal rexnord corporation manufactures sells i...
1,APTMU,Alpha Partners Technology Merger Corp.,10.6200,0.013,0.311,0.000021,Alpha Partners Technology Merger Corp. does no...,alpha partners technology merger corp signific...
2,WSFS,WSFS Financial Corporation,35.2350,9.927,0.047,2.967547,WSFS Financial Corporation operates as the sav...,wsfs financial corporation operates savings lo...
3,FRAF,Franklin Financial Services Corporation,32.0000,6.153,-0.162,0.961493,Franklin Financial Services Corporation operat...,franklin financial services corporation operat...
4,WF,Woori Financial Group Inc.,26.7500,63314.470,-0.342,121665.258441,"Woori Financial Group Inc., together with its ...",woori financial group inc together subsidiarie...
...,...,...,...,...,...,...,...,...
230,WEYS,"Weyco Group, Inc.",28.5300,2.452,0.064,0.225463,"Weyco Group, Inc. designs and distributes foot...",weyco group inc designs distributes footwear m...
231,FE,FirstEnergy Corp.,36.0085,0.206,0.196,0.001411,"FirstEnergy Corp., through its subsidiaries, g...",firstenergy corp subsidiaries generates transm...
232,SHEL,Shell plc,65.3350,6.695,-0.808,0.121649,Shell plc operates as an energy and petrochemi...,shell plc operates energy petrochemical compan...
233,TXT,Textron Inc.,75.4400,8.525,0.274,1.266532,"Textron Inc. operates in the aircraft, defense...",textron inc operates aircraft defense industri...


In [13]:
company_data.to_csv("/content/drive/MyDrive/SEM9/Information Retrieval/Financial_Search_Engine/package datasets/Company_Data_Preprocessed")

In [14]:
def extract_keywords_from_documents(documents, top_n=10):
    # Combine documents into a single string
    text = " ".join(documents)

    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Convert the filtered words back into a string
    filtered_text = " ".join(filtered_words)

    # Use TF-IDF to extract keywords
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([filtered_text])
    feature_names = vectorizer.get_feature_names_out()
    dense = tfidf_matrix.todense()

    # Get the top N keywords
    keywords = [feature_names[i] for i in dense.argsort(axis=1).tolist()[0][-top_n:]]
    return keywords

In [15]:
top_keywords = extract_keywords_from_documents(process_docs, top_n=300)
print("Top Keywords:", top_keywords)

Top Keywords: ['beverage', 'hotel', 'december', 'restaurants', 'electronics', 'institutions', 'engineering', 'installment', 'logistics', 'ingredients', 'installation', 'debit', 'thirdparty', 'process', 'four', 'monitoring', 'washington', 'machines', 'administration', 'us', 'kingdom', 'networks', 'video', 'gaming', 'partners', 'invests', 'agricultural', 'motors', 'storage', '2021', 'time', 'trucks', 'land', 'utility', 'merger', 'virginia', 'clients', 'resources', 'consulting', 'aerospace', 'demand', 'semiconductor', 'focuses', 'delivery', 'secured', 'planning', 'property', '2022', 'cleaning', 'sale', 'programs', 'suite', 'activities', 'community', 'mexico', 'travel', 'beverages', 'contract', 'combination', 'involved', 'national', 'retirement', 'retailers', 'media', 'original', 'local', 'marketing', 'information', 'merchandise', 'consumers', 'transfer', 'china', 'advisory', 'automated', 'certificates', 'medical', 'ecommerce', 'freight', 'multifamily', 'cards', 'individual', 'payment', 'm

In [16]:
with open("/content/drive/MyDrive/SEM9/Information Retrieval/Financial_Search_Engine/package datasets/keywords.txt", "w") as output:
    output.write(str(top_keywords))

#### Boolean Model

In [17]:
inv_index = {key:[] for key in top_keywords}
for i in range(len(process_docs)):
    for w in top_keywords:
        strs = [' ' + w, w + ' ', w + 's', w + "'"]
        if strs[0] in process_docs[i] or strs[1] in process_docs[i] or strs[2] in process_docs[i] or strs[3] in process_docs[i]:
            if i not in inv_index[w]:
                inv_index[w].append(i)
for k,v in inv_index.items():
    print(k , '- no of occurrences = ', len(v))

beverage - no of occurrences =  13
hotel - no of occurrences =  9
december - no of occurrences =  14
restaurants - no of occurrences =  9
electronics - no of occurrences =  8
institutions - no of occurrences =  11
engineering - no of occurrences =  13
installment - no of occurrences =  14
logistics - no of occurrences =  11
ingredients - no of occurrences =  4
installation - no of occurrences =  10
debit - no of occurrences =  15
thirdparty - no of occurrences =  14
process - no of occurrences =  32
four - no of occurrences =  18
monitoring - no of occurrences =  11
washington - no of occurrences =  11
machines - no of occurrences =  10
administration - no of occurrences =  15
us - no of occurrences =  125
kingdom - no of occurrences =  15
networks - no of occurrences =  10
video - no of occurrences =  13
gaming - no of occurrences =  5
partners - no of occurrences =  16
invests - no of occurrences =  12
agricultural - no of occurrences =  12
motors - no of occurrences =  6
storage - n

In [18]:
inv_index

{'beverage': [0, 13, 54, 109, 122, 129, 150, 155, 162, 170, 172, 193, 216],
 'hotel': [13, 18, 47, 67, 94, 134, 159, 160, 170],
 'december': [21, 53, 57, 62, 73, 93, 109, 126, 128, 161, 163, 178, 208, 229],
 'restaurants': [13, 50, 67, 155, 162, 170, 172, 183, 223],
 'electronics': [32, 58, 71, 98, 109, 148, 167, 216],
 'institutions': [30, 41, 56, 170, 171, 174, 188, 196, 207, 211, 219],
 'engineering': [18, 32, 58, 65, 117, 118, 132, 161, 165, 167, 185, 214, 234],
 'installment': [2, 3, 4, 24, 41, 44, 76, 90, 146, 184, 206, 213, 225, 228],
 'logistics': [2, 16, 48, 65, 77, 109, 117, 151, 161, 166, 211],
 'ingredients': [77, 141, 193, 224],
 'installation': [18, 50, 78, 99, 108, 136, 140, 161, 166, 182],
 'debit': [4, 5, 41, 49, 76, 90, 103, 146, 153, 184, 192, 217, 219, 221, 225],
 'thirdparty': [13, 47, 48, 53, 58, 67, 72, 78, 83, 86, 118, 122, 140, 172],
 'process': [2,
  15,
  16,
  18,
  20,
  21,
  23,
  32,
  70,
  73,
  75,
  92,
  99,
  105,
  129,
  130,
  131,
  141,
  145,

In [19]:
def get_set(word):
    if word in inv_index:
        return set(inv_index[word])
    else:
        return set()

In [46]:
def bool_eval(query):
    query_words = query.split(' ')
    query_words = [i for i in query_words if i in top_keywords]
    docs = [set(inv_index[w]) for w in query_words]
    result = list(set.intersection(*docs))
    output=[]
    print("Matched Companies: ")
    for i in range(len(result)):
      output.append(company_symbols[result[i]])
      print(company_symbols[result[i]], '  ', process_docs[result[i]])
      #print('Document {:02d} : '.format(i+1), process_docs[result[i]])
    return output

In [73]:
query_bool = 'finance'
bool_results=bool_eval(query_bool)

Matched Companies: 
JOB    gee group inc provides permanent temporary professional industrial staffing placement services united states company operates two segments industrial staffing services professional staffing services offers placement information technology accounting finance office engineering medical professionals direct hire contract staffing services temporary staffing services light industrial clients company also provides electronic medical record services emergency departments specialty physician practices clinics offers professional commercial staffing services names access data consulting agile resources ashley ellis gee group omni one paladin consulting triad hospital emergency rooms physician practice services scribe solutions brand contract direct hire professional staffing services accounting staffing sni banking sni certes sni energy sni financial sni technology brands company formerly known general employment enterprises inc changed name gee group inc july 2016 g

#### Vector Space Models


In [25]:
def cosine_sim(query, X):
    q_tfidf = vectorizer.fit_transform([query])
    results = cosine_similarity(X,q_tfidf).reshape((-1,))
    return results

In [48]:
def query_search(results):
    print("Top 10 companies matched: ")
    j = 0
    output=[]
    for i in results.argsort()[-10:][::-1]:
        j += 1
        output.append(company_symbols[i])
        print(company_symbols[i], '  ', process_docs[i])
    return output

In [27]:
vectorizer = TfidfVectorizer(vocabulary=top_keywords)
X = vectorizer.fit_transform(process_docs)
print(X.shape)

(235, 300)


In [74]:
query_vm = 'agriculture'
vector_model_results=query_search(cosine_sim(query_vm, X))

Top 10 companies matched: 
CSIQ    canadian solar inc together subsidiaries designs develops manufactures sells solar ingots wafers cells modules solar power battery storage products asia americas europe internationally company operates two segments canadian solar inc csi solar global energy csi solar segment offers standard solar modules battery storage solutions well solar system kits readytoinstall packages comprising inverters racking systems accessories engineering procurement construction epc services global energy segment engages development construction maintenance sale solar battery storage projects operation solar power plants sale electricity segment also provides operation maintenance om services including monitoring inspections repair replacement plant equipment site management administrative support services solar projects well asset management services january 31 2023 segment fleet solar power plants operation aggregate capacity approximately 574 mwp company serves distr

#Ranking the results

In [59]:
def rank_stocks_by_intrinsic_value(stock_list):
    stock_list.sort(key=lambda stock: company_data[company_data['Symbol'] == stock]['Intrinsic_Value'].iloc[0], reverse=True)
    ranked_Output=defaultdict()
    for stock in stock_list:
      ranked_Output[company_data[company_data['Symbol'] == stock]['Company_Name'].iloc[0]]=company_data[company_data['Symbol'] == stock]['Long Business Summary'].iloc[0]
    return ranked_Output

In [75]:
print("Ranked Output for boolean model\nQuery: ",query_bool,"\n")
bool_model_ranks=rank_stocks_by_intrinsic_value(bool_results)
i=0
for company_name,summary in bool_model_ranks.items():
  print(i+1,".", company_name)
  print(summary)
  print("\n")
  i+=1

Ranked Output for boolean model
Query:  finance 

1 . Deutsche Bank Aktiengesellschaft
Deutsche Bank Aktiengesellschaft, operates as a stock corporation, engages in the provision of corporate and investment banking, and asset management products and services to private clients, corporate entities, and institutional clients worldwide. Its Corporate Bank segment provides cash management, trade finance and lending, trust and agency, foreign exchange, and securities services, as well as risk management solutions. The company's Investment Bank segment offers debt origination, merger and acquisitions, and equity advisory services. Its Private Bank segment provides payment and account services, and credit and deposit products, as well as investment advice products, such as environmental, social, and governance products. This segment also provides banking, wealth management services, postal and parcel services; and offers support in planning, managing and investing wealth, financing personal a

In [76]:
print("Ranked Output for boolean model\nQuery: ",query_vm,"\n")

vs_model_ranks=rank_stocks_by_intrinsic_value(vector_model_results)
i=0
for company_name,summary in vs_model_ranks.items():
  print(i+1,".", company_name)
  print(summary)
  print("\n")
  i+=1

Ranked Output for boolean model
Query:  agriculture 

1 . Universal Display Corporation
Universal Display Corporation engages in the research, development, and commercialization of organic light emitting diode (OLED) technologies and materials for use in display and solid-state lighting applications. The company offers PHOLED technologies and materials for displays and lighting products under the UniversalPHOLED brand. It is also involved in the research, development, and commercialization of other OLED device and manufacturing technologies, including FOLED that are flexible OLEDs for the fabrication of OLEDs on flexible substrates; and OVJP, an organic vapor jet printing technology. In addition, the company provides technology development and support services, including third-party collaboration and support to third parties for the commercialization of their OLED products; and contract research services in the areas of chemical materials synthesis research, development, and commercial